<a href="https://colab.research.google.com/github/payaljoshi-mscit/sentiment-analysis/blob/main/tc2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install and Import NLTK
Terminal $  pip install nltk==3.3

In [2]:
import nltk  #python console
# https://www.nltk.org/
nltk.download('twitter_samples') #python console  #Download tweeets locally


[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

Load data 

In [3]:
from nltk.corpus import twitter_samples # import data in program
positive_tweets = twitter_samples.strings('positive_tweets.json')  #5000 positive tweets
negative_tweets = twitter_samples.strings('negative_tweets.json')  #5000 negative tweets
#text = twitter_samples.strings('tweets.20150430-223406.json')      #20000 tweets with no sentiments


Download Tokenizer

In [4]:
nltk.download('punkt') #python console , pre-trained tokenizer

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
print(tweet_tokens[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']


In [5]:
nltk.download('wordnet')  #python console https://www.nltk.org/howto/wordnet.html
nltk.download('averaged_perceptron_tagger') #python console pre-trained POS tagger

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [6]:
from nltk.tag import pos_tag
print(pos_tag(tweet_tokens[0]))


[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [7]:
nltk.download('omw-1.4') #python console , wordnet multi-lingual data corpus


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
# Before running a lemmatizer, you need to determine the context for each word in your text.
from nltk.stem.wordnet import WordNetLemmatizer
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))


['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


Remove Noise

In [9]:
import re, string
# Remove hyperlinks, punctuations, twitter handles with @ 
# Remove stop words
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [10]:
nltk.download('stopwords') #python console


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))


['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [12]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])


['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


Get word frequency

In [13]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10)) #The .most_common() method lists the words which occur most frequently in data. 


[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


Prepare data for model training 

In [14]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)  #convert cleaned tokens and generates dictionary data format

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]  #7000 for training
test_data = dataset[7000:]   #3000 for testing 


Build and Test Model

In [15]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data) # Model generation
print(classifier.labels() )
print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))
# ratio of occurrence of a token in positive and negative tagged tweets in the training dataset

['Positive', 'Negative']
Accuracy is: 0.9953333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2073.1 : 1.0
                follower = True           Positi : Negati =     22.3 : 1.0
                     bam = True           Positi : Negati =     20.4 : 1.0
                  arrive = True           Positi : Negati =     19.9 : 1.0
                     sad = True           Negati : Positi =     19.4 : 1.0
               community = True           Positi : Negati =     15.7 : 1.0
                 welcome = True           Positi : Negati =     14.2 : 1.0
                     x15 = True           Negati : Positi =     13.6 : 1.0
           unfortunately = True           Negati : Positi =     12.3 : 1.0
                    miss = True           Negati : Positi =     12.3 : 1.0
None


In [16]:
from nltk.tokenize import word_tokenize

custom_tweet = "Machine learning is cool and awesome."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))


Positive
